In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
from sklearn.metrics import f1_score

In [2]:
# Loading the train data
df = pd.read_csv('./train.csv')
X = df
y = df['Education']


In [3]:
X = X.drop('Candidate',axis=1)
X = X.drop('Constituency ∇',axis=1)
X.drop('ID',axis=1,inplace=True)
for row in X.index:
    #Remove WhiteSpaces in State in the middle of the string.
    X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
    # print(X.loc[row, "Total Assets"][-4:])
    if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
    elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
        X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
        X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
    elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
        X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
        X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
    elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
        X.loc[row,  "Total Assets"] = "0"
for row in X.index:
    if(isinstance(X.loc[row, "Liabilities"], float)):
        X.loc[row, "Liabilities"] = "0"
    elif(X.loc[row, "Liabilities"][-4:] == "Lac+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
    elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
    elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
        X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
        X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
    elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
        X.loc[row, "Liabilities"] = "0"
# Mapping Parties to unique values in the dataframe using map function
# Create a dictionary of the parties and their corresponding unique values
party = X['Party'].unique()
party = {value: idx for idx, value in enumerate(party)}
# Add a value for nan parties
party[""] = -1
# Mapping the values to the dataframe
X['Party'] = X['Party'].map(party)
state = X['state'].unique()
state = {value: idx for idx, value in enumerate(state)}
# Mapping the values to the dataframe
X['state'] = X['state'].map(state)
education = y.unique()
education = {value: idx for idx, value in enumerate(education)}
# Mapping the values to the dataframe
y = y.map(education)
X['Education'] = y

In [4]:
# Synthetic Data
df = pd.read_csv('./synthetic_data_best.csv')
X_Synthetic = df
X_Synthetic['state'] = X_Synthetic['state'].map(state)
X_Synthetic['Party'] = X_Synthetic['Party'].map(party)
X_Synthetic['Education'] = X_Synthetic['Education'].map(education)

In [5]:
# Append the synthetic data to the original data
df_combined = pd.concat([X, X_Synthetic], ignore_index=True)
X = df_combined
X_Train_EducationWise = []
Y_Train_EducationWise = []
for i in range(len(education)):
	# Change all values in education except i to 0
	y = X['Education']
	y = y.apply(lambda x: 1 if x == i else 0)
	Y_Train_EducationWise.append(y)
	X_Train_EducationWise.append(X.drop('Education', axis = 1))

Y = X['Education']
X = X.drop('Education', axis = 1)

In [6]:
def test_preprocessing(X, state, party):
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	return X

def answer_preprocessing(X, state, party, education):
	X = X.drop('Candidate',axis=1)
	X = X.drop('Constituency ∇',axis=1)
	X.drop('ID',axis=1,inplace=True)
	for row in X.index:
		X.loc[row, "state"] = X.loc[row, "state"].replace(" ", "")
		# print(X.loc[row, "Total Assets"][-4:])
		if(X.loc[row, "Total Assets"][-4:] == "Lac+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-4]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 100)
		elif(X.loc[row, "Total Assets"][-6:] == "Crore+"):
			X.loc[row, "Total Assets"] = X.loc[row, "Total Assets"][:-6]
			X.loc[row, "Total Assets"] = str(int(X.loc[row, "Total Assets"]) * 10000)
		elif(X.loc[row,  "Total Assets"][-5:] == "Thou+"):
			X.loc[row,  "Total Assets"] = X.loc[row,  "Total Assets"][:-5]
			X.loc[row,  "Total Assets"] = str(int(X.loc[row,  "Total Assets"]))
		elif(X.loc[row,  "Total Assets"][-5:] == "Hund+"):
			X.loc[row,  "Total Assets"] = "0"
	for row in X.index:
		if(X.loc[row, "Liabilities"][-4:] == "Lac+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-4]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 100)
		elif(X.loc[row, "Liabilities"][-6:] == "Crore+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-6]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]) * 10000)
		elif(X.loc[row, "Liabilities"][-5:] == "Thou+"):
			X.loc[row, "Liabilities"] = X.loc[row, "Liabilities"][:-5]
			X.loc[row, "Liabilities"] = str(int(X.loc[row, "Liabilities"]))
		elif(X.loc[row, "Liabilities"][-5:] == "Hund+"):
			X.loc[row, "Liabilities"] = "0"
	X['Party'] = X['Party'].map(party)
	X['state'] = X['state'].map(state)
	X['Education'] = X['Education'].map(education)
	return X

# Answers

In [7]:
answer = pd.read_csv('./answers.csv')
answer['Education'] = answer['Education'].map(education)

def calculate_accuracy(Y_Test_StateWise, answer_statewise):
	#convert this to an array
    temp = np.array(answer_statewise['Education'])
    correct_count = 0
    for i in range(len(Y_Test_StateWise)):
        if(temp[i] == Y_Test_StateWise[i]):
            correct_count += 1
    f1 = f1_score(Y_Test_StateWise, temp, average='weighted')
    return [correct_count/len(Y_Test_StateWise) * 100, correct_count, f1]



In [8]:
X_Test_EducationWise = []
Y_Test_EducationWise = []
answer = answer_preprocessing(pd.read_csv('./answers.csv'), state, party, education)
X_Test = answer
for i in range(len(education)):
	# Change all values in education except i to 0
	y = X_Test['Education']
	y = y.apply(lambda x: 1 if x == i else 0)
	Y_Test_EducationWise.append(y)
	X_Test_EducationWise.append(X_Test.drop('Education', axis = 1))

In [9]:
X_Test_EducationWise[0]

,Party,Criminal Case,Total Assets,Liabilities,state
0,8,2,700000,110000,7
1,3,1,20000,1300,4
2,14,3,490000,10000,7
3,3,1,20000,0,4
4,5,0,160000,20000,5
...,...,...,...,...,...
1369,1,0,10000,0,4
1370,7,1,0,1300,8
1371,2,1,50000,0,3
1372,1,1,20000,500,9


In [10]:
Y_Test_EducationWise[0]

0       0
1       0
2       0
3       0
4       0
       ..
1369    0
1370    0
1371    0
1372    0
1373    0
Name: Education, Length: 1374, dtype: int64

In [11]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1369    0
1370    0
1371    0
1372    0
1373    0
Name: Education, Length: 1374, dtype: int64

In [12]:
X_Test

,Party,Criminal Case,Total Assets,Liabilities,state,Education
0,8,2,700000,110000,7,1
1,3,1,20000,1300,4,5
2,14,3,490000,10000,7,5
3,3,1,20000,0,4,3
4,5,0,160000,20000,5,2
...,...,...,...,...,...,...
1369,1,0,10000,0,4,1
1370,7,1,0,1300,8,4
1371,2,1,50000,0,3,4
1372,1,1,20000,500,9,2


# KNN EducationWise

In [16]:
#Implement KNN
from sklearn.neighbors import KNeighborsClassifier

models_KNN = []
Y_Test_EducationWise_Answer = []
neighbours_calculated = dict()
f1_KNN = dict()

# Grid Search
for i in range(len(education)):
	print(i)
	f1 = []
	for j in range(1, 15):
		model = KNeighborsClassifier(n_neighbors=j)
		model.fit(X_Train_EducationWise[i], Y_Train_EducationWise[i])
		y = model.predict(X_Test_EducationWise[i])
		t = calculate_accuracy(y, X_Test)[2]
		f1.append(t)
	neighbours_calculated[i] = (f1.index(max(f1)) + 1)
	model = KNeighborsClassifier(n_neighbors=f1.index(max(f1)) + 1)
	model.fit(X_Train_EducationWise[i], Y_Train_EducationWise[i])
	models_KNN.append(model)
	Y_Test_EducationWise_Answer.append(model.predict(X_Test_EducationWise[i]))
	f1 = calculate_accuracy(Y_Test_EducationWise_Answer[i], X_Test)[2]
	f1_KNN[i] = f1

0
1
2
3
4
5
6
7
8
9


In [17]:
print(f1_KNN)

{0: 0.0702247191011236, 1: 0.08114604660051582, 2: 0.09913318505343738, 3: 0.0803707267329501, 4: 0.11560272013994456, 5: 0.07596983631952048, 6: 0.0706905176633732, 7: 0.07021571457440248, 8: 0.07061803354217208, 9: 0.0702512292302944}


In [21]:
# Get KNN Predictions Probabilities
Y_Test_EducationWise_Answer_Prob = []
for i in range(len(education)):
	Y_Test_EducationWise_Answer_Prob.append(models_KNN[i].predict_proba(X_Test_EducationWise[i]))

Y_Test_EducationWise_Answer_Prob[4]

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])